Importing Libraries

In [ ]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio 
import sklearn 
from sklearn.model_selection import train_test_split, cross_val_score 
from sklearn.ensemble import RandomForestClassifier  
from sklearn.metrics import accuracy_score, confusion_matrix  
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder  
from bayes_opt import BayesianOptimization
import optuna
pio.templates.default = "plotly_white"  
import joblib

Loading and Viewing The Data

In [ ]:
credit_card_data = pd.read_csv("train.csv") 
print(credit_card_data.info())

In [ ]:
# Checking if dataset has any null values or not 
print(credit_card_data.isnull().sum()) # No null values for any columns ; no need for imputation

Data Visualization

In [ ]:
fig = px.box(credit_card_data, x = "Occupation", color = "Credit_Score", title = "Credit Scores Based on Occupation") 
fig.show() # No difference in credit scores among different occupations 

In [ ]:
fig = px.box(credit_card_data, x = "Credit_Score", y = "Annual_Income", color = "Credit_Score", title = "Credit Scores Based on Annual Income")
fig.show() # Average annual income increases as credit score increases 

In [ ]:
fig = px.box(credit_card_data, x = "Credit_Score", y = "Age", color = "Credit_Score", title = "Credit Scores Based on Annual Income")
fig.show() # Older people tend to have higher average credit scores 

In [ ]:
fig = px.box(credit_card_data, x = "Credit_Score", y = "Monthly_Inhand_Salary", color = "Credit_Score", title = "Credit Scores Based on Monthly Inhand Salary")
fig.show() # The more monthly inhand salary you receive, the higher your credit score will most likely be 

In [ ]:
fig = px.box(credit_card_data, x = "Credit_Score", y = "Num_Bank_Accounts", color = "Credit_Score", title = "Credit Scores Based on Number of Bank Accounts")
fig.show() # Having a large amount of bank accounts negatively impacts your credit score; 2-3 bank accounts are an optimal number 

In [ ]:
fig = px.box(credit_card_data, x = "Credit_Score", y = "Num_Credit_Card", color = "Credit_Score", title = "Credit Scores Based on Number of Credit Cards")
fig.show() # Having a large amount of credit card accounts is bad for your credit score 

In [ ]:
fig = px.box(credit_card_data, x = "Credit_Score", y = "Interest_Rate", color = "Credit_Score", title = "Credit Scores Based on Interest Rate")
fig.show() # Account holders with good credit scores are offered lower interest rates on average

In [ ]:
fig = px.box(credit_card_data, x = "Credit_Score", y = "Num_of_Loan", color = "Credit_Score", title = "Credit Scores Based on Number of Loans")
fig.show() # People with good credit scores tend to take a lower amount of loans 

In [ ]:
fig = px.box(credit_card_data, x = "Type_of_Loan", color = "Credit_Score", title = "Credit Scores Based on Type of Loan Taken")
fig.show() # No real correlation to credit score & the type of loan taken 

In [ ]:
fig = px.box(credit_card_data, x = "Credit_Score", y = "Delay_from_due_date", color = "Credit_Score", title = "Credit Scores Based on Avg Number of Days Delayed for Credit Card Payments")
fig.show() # People with lower number of days delayed for credit card have better credit scores 

In [ ]:
fig = px.box(credit_card_data, x = "Credit_Score", y = "Num_of_Delayed_Payment", color = "Credit_Score", title = "Credit Scores Based on Number of Delayed Card Payments")
fig.show() # People with lower number of delayed payments have higher credit scores(4-12 late payments are fine)

In [ ]:
fig = px.box(credit_card_data, x = "Credit_Score", y = "Changed_Credit_Limit", color = "Credit_Score", title = "Credit Scores Based on % Change in Credit Card Limit")
fig.show() # People with good credit scores have lower percent change in credit card limit(lower correlation compared to other indicators) 

In [ ]:
fig = px.box(credit_card_data, x = "Credit_Score", y = "Num_Credit_Inquiries", color = "Credit_Score", title = "Credit Scores Based on Number of Credit Card Inquiries")
fig.show() # People with good credit scores have lower number of credit card inquiries(lots of outliers for good credit scores)

In [ ]:
fig = px.box(credit_card_data, x = "Credit_Score", y = "Outstanding_Debt", color = "Credit_Score", title = "Credit Scores Based on Outstanding Debt")
fig.show() # People with lower outstanding debt have higher credit scores(lots of outliers)

In [ ]:
fig = px.box(credit_card_data, x = "Credit_Score", y = "Credit_Utilization_Ratio", color = "Credit_Score", title = "Credit Scores Based on Credit Utilization Ratio")
fig.show() # No effect on credit scores

In [ ]:
fig = px.box(credit_card_data, x = "Credit_Score", y = "Total_EMI_per_month", color = "Credit_Score", title = "Credit Scores Based on Total EMI Per Month")
fig.show() # Low impact on credit score 

In [ ]:
fig = px.box(credit_card_data, x="Credit_Score", y="Amount_invested_monthly", color="Credit_Score", title="Credit Scores Based on Amount Invested Monthly") 
fig.show() # On average, people with higher credit scores have invested more money each month(lots of outliers)

In [ ]:
fig = px.box(credit_card_data, x="Credit_Score", y="Monthly_Balance", color="Credit_Score", title="Credit Scores Based on Monthly Balance") 
fig.show() # People with higher monthly balances have higher credit scores 


Creating a Training, Validation, & Test Set

In [ ]:
train_df, test_df = train_test_split(credit_card_data,test_size=0.33,random_state=42)

print('train_df.shape :', train_df.shape)
print('test_df.shape :', test_df.shape)

In [ ]:
input_cols = ["Age", "Annual_Income", "Monthly_Inhand_Salary", "Num_Bank_Accounts", "Num_Credit_Card","Interest_Rate","Num_of_Loan","Delay_from_due_date","Changed_Credit_Limit", "Num_Credit_Inquiries", "Credit_Mix", "Credit_History_Age", "Amount_invested_monthly", "Monthly_Balance"]
target_col = ["Credit_Score"]

In [ ]:
train_inputs = train_df[input_cols].copy() 
train_targets = train_df[target_col].copy() 

test_inputs = test_df[input_cols].copy() 
test_targets = test_df[target_col].copy() 


In [ ]:
numeric_cols = train_inputs.select_dtypes(include = np.number).columns.tolist()
categorical_cols = train_inputs.select_dtypes("object").columns.tolist()  


Scaling & One-Hot Encoding 

In [ ]:
scaler = MinMaxScaler().fit(credit_card_data[numeric_cols]) 
train_inputs[numeric_cols] = scaler.transform(train_inputs[numeric_cols])
test_inputs[numeric_cols] = scaler.transform(test_inputs[numeric_cols])  


In [ ]:
encoder = OneHotEncoder(sparse_output=False, handle_unknown="ignore").fit(credit_card_data[categorical_cols])
encoded_cols = list(encoder.get_feature_names_out(categorical_cols))

train_inputs[encoded_cols] = encoder.transform(train_inputs[categorical_cols])
test_inputs[encoded_cols] = encoder.transform(test_inputs[categorical_cols])

In [ ]:
x_train = train_inputs[numeric_cols + encoded_cols]
x_test = test_inputs[numeric_cols + encoded_cols] 
x_train

Training the Baseline Model

In [ ]:
baseline_model = RandomForestClassifier(n_jobs = -1, random_state = 42).fit(x_train, train_targets) 
train_acc = accuracy_score(train_targets,baseline_model.predict(x_train)) 
test_acc = accuracy_score(test_targets, baseline_model.predict(x_test)) 
print(train_acc, test_acc) # Training Accuracy = 0.9999, Test Accuracy = 0.8099

Hyperparameter Tuning Using Optuna,Bayesian Optimization

In [ ]:
def objective(trial):
    n_estimators = trial.suggest_int('n_estimators', 10, 100)
    max_depth = trial.suggest_int('max_depth', 2, 32)
    min_samples_split = trial.suggest_int('min_samples_split', 2, 10)
    min_samples_leaf = trial.suggest_int('min_samples_leaf', 1, 10)
    max_features = trial.suggest_categorical('max_features', ['sqrt', 'log2', None])  # Removed 'auto'

    model = RandomForestClassifier(
        n_estimators=n_estimators,
        max_depth=max_depth,
        min_samples_split=min_samples_split,
        min_samples_leaf=min_samples_leaf,
        max_features=max_features,
        random_state=42
    )
    
    score = cross_val_score(model, x_train, train_targets, n_jobs=-1, cv=10, scoring='accuracy').mean()
    
    return score

# Create the Optuna study and optimize
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=20)

# Print the best hyperparameters and picking final model 
print("Best hyperparameters: ", study.best_params) 


In [ ]:
final_rf_model = RandomForestClassifier(n_jobs=-1, random_state=42, **study.best_params).fit(x_train, train_targets)
train_acc = accuracy_score(train_targets,final_rf_model.predict(x_train)) 
test_acc = accuracy_score(test_targets, final_rf_model.predict(x_test)) 
print(train_acc,test_acc) # 0.89, 0.80 (less overfitting)